<a href="https://colab.research.google.com/github/Lnaden/iqb-2025/blob/main/7LME_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Docking with gnina

This notebook shows a full pipeline for docking with gnina including my analysis and reasoning for docking choices.

For this notebook, we perform cognate docking in order to determine performance of gnina and other ideal input conditions.

## Set Up

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
!ls

condacolab_install.log	sample_data


In [ ]:
%%capture output
!conda install conda-forge::mdanalysis conda-forge::pdbfixer conda-forge::openmm conda-forge::openmmforcefields

In [ ]:
%%capture output
!apt install openbabel

In [ ]:
%%capture output
!pip install useful_rdkit_utils py3Dmol

In [ ]:
%%capture output
!wget https://github.com/gnina/gnina/releases/download/v1.3/gnina

In [ ]:
!chmod +x gnina

## Get Files

In [ ]:
import os
import requests

pdb_id = "7LME"
ligand_id = "Y6J" # This is the ligand bound to the structure.

In [ ]:


# --- Protein Download ---
protein_directory = "protein_structures"
protein_filename = f"{pdb_id}.pdb"
protein_filepath = os.path.join(protein_directory, protein_filename)

os.makedirs(protein_directory, exist_ok=True)

print(f"Downloading protein {pdb_id}...")
protein_url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
protein_request = requests.get(protein_url)
protein_request.raise_for_status() # Check for errors

with open(protein_filepath, "w") as f:
    f.write(protein_request.text)
print(f"Saved protein to {protein_filepath}")

# --- Ligand Download ---
# we are downloading an "ideal" version of the ligand
# that we will use to make sure our posed ligand
# has the correct chemistry (since)
ligand_directory = "ligand_structures"
ligand_filename = f"{ligand_id}_ideal.sdf"
ligand_filepath = os.path.join(ligand_directory, ligand_filename)

os.makedirs(ligand_directory, exist_ok=True)

print(f"Downloading ligand {ligand_id}...")
ligand_url = f"https://files.rcsb.org/ligands/download/{ligand_filename}"
ligand_request = requests.get(ligand_url)
ligand_request.raise_for_status() # Check for errors

with open(ligand_filepath, "w") as f:
    f.write(ligand_request.text)
print(f"Saved ligand to {ligand_filepath}")

Saved protein to protein_structures/7LME.pdb
Saved ligand to ligand_structures/Y6J_ideal.sdf


## Protein Prep and Analysis

* Add hydrogens for ph.
* Use PDB Fixer to add missing residues.
* Perform energy minimization with OpenMM.

In [ ]:
from pdbfixer import PDBFixer
from openmm.app import PDBFile, Simulation, ForceField
from openmm import Platform, VerletIntegrator
from openmm import unit

# Fix structure using PDBFixer
fixer = PDBFixer(filename='protein_structures/7LME.pdb')
forcefield = ForceField("amber/protein.ff14SB.xml")

fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(keepWater=False)
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(forcefield=forcefield)

# Create OpenMM system for minimization
system = forcefield.createSystem(fixer.topology)

# Use a generic VerletIntegrator
integrator = VerletIntegrator(0.001 * unit.picoseconds)

# Create simulation for minimization
platform = Platform.getPlatformByName('CUDA')
simulation = Simulation(fixer.topology, system, integrator, platform)
simulation.context.setPositions(fixer.positions)

# Minimize energy
print('Minimizing energy...')
simulation.minimizeEnergy()

# Get minimized positions
minimized_positions = simulation.context.getState(getPositions=True).getPositions()

# Write minimized structure to a PDB file
with open('protein_structures/7LME_fixed.pdb', 'w') as output:
    PDBFile.writeFile(fixer.topology, minimized_positions, output)

print('Minimization complete. Minimized structure saved to protein_structures/7LME_fixed.pdb.')

Minimizing energy...
Minimization complete. Minimized structure saved to protein_structures/7LME_fixed.pdb.


## Ligand Structure from PDB

To do RMSD analysis later from the crystal structure, I need to get the ligand in its protein pose with the correct bonding information.

We downloaded the "ideal" ligand from the PDB. We will use this as a reference to make sure the molecule information is correct, and then match with the pose structure from the protein.

In [ ]:
# First isolate the ligand using MDanalysis

import MDAnalysis as mda
import numpy as np

from rdkit import Chem
from rdkit.Chem import AllChem

u = mda.Universe(f"{protein_directory}/{pdb_id}.pdb")

protein = u.select_atoms("protein")
ligand = u.select_atoms(f"resname {ligand_id} and segid A")

ligand.write(f"{ligand_directory}/{ligand_id}_fromPDB.pdb")


/usr/local/lib/python3.11/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem

ideal_mol = Chem.MolFromMolFile(f"{ligand_directory}/{ligand_id}_ideal.sdf", removeHs=True)
pose_mol = Chem.MolFromPDBFile(f"{ligand_directory}/{ligand_id}_fromPDB.pdb", removeHs=True)

# Assign bond orders from the template to the pose molecule
corrected_pose = AllChem.AssignBondOrdersFromTemplate(ideal_mol, pose_mol)

# Add hydrogens back to the corrected pose
corrected_pose_with_H = Chem.AddHs(corrected_pose, addCoords=True)

# Make sure the molecule is right (check smiles of both)
assert Chem.MolToSmiles(corrected_pose) == Chem.MolToSmiles(ideal_mol)

# Save the corrected pose to an SDF file
writer = Chem.SDWriter(f"{ligand_directory}/{ligand_id}_corrected_pose.sdf")
writer.write(corrected_pose_with_H)
writer.close()


[21:47:30] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[21:47:30] WARNING: More than one matching pattern found - picking one



## Flexible Docking?

[This paper](https://pubs.acs.org/doi/10.1021/acsomega.2c07259) says that the binding site of Mpro (main protease) is flexible but cites a source for this that I haven't looked into yet.

## Run gnina

Run gnina. This section can be used to create an input script to run gnina with several different parameters.

Then, analyze the RMSD using from the crystal structure and see what is the closest to the original ligand. RMSD calculation uses obrms.

This is to assess exhaustiveness, rigid vs. flexible docking, and scoring function. How well do the predicted poses align with the bound pose?

Docking poses will be in folder `docking_results_{label}` and RMSD measurements for the docking poses vs. the references structure are in `rmsd_outputs_{label}`. The variable `{label}` is defined in the cell below (default nothing).

In [ ]:
exhaustiveness_levels = [12]
modes = ["flex", "rigid"]
scoring_functions = ["cnn"] # cnn, vinardo, no more specific choosing of cnn currently/
label = "" # additional label to append to the end of the output directory names if running more than one trial (execute cell again with new parameters)

# File paths
docked_dir = f"docking_results_{label}"
ligand_ideal = f"{ligand_directory}/{ligand_id}_ideal.sdf"
ligand_pose = f"{ligand_directory}/{ligand_id}_corrected_pose.sdf"
pdb_path = f"{protein_directory}/{pdb_id}_fixed.pdb"
rmsd_directory = f"rmsd_outputs_{label}"

# this is needed for space_groups.txt. Gnina is not able to find this
# without us setting it when run from a script as we are doing.
babel_datadir = "/usr/share/openbabel/3.1.1"


os.makedirs(rmsd_directory, exist_ok=True)
os.makedirs(docked_dir, exist_ok=True)

bash_lines = ["#!/bin/bash\n\n"]

bash_lines.append(f"export BABEL_DATADIR={babel_datadir}\n")


for scoring_function in scoring_functions:
  for ex in exhaustiveness_levels:
      for mode in modes:
          out_sdf = f"{docked_dir}/{ligand_id}_e{ex}_{mode}_{scoring_function}.sdf"
          rmsd_txt = f"rmsd_e{ex}_{mode}_{scoring_function}.txt"

          # temp - comment out rigid docking with cnn
          #if mode == "rigid" and scoring_function == "cnn":
          #    continue

          gnina_cmd = f"./gnina -r {pdb_path} -l {ligand_ideal} --autobox_ligand {ligand_pose}"

          if mode == "flex":
              gnina_cmd += f" --flexdist_ligand {ligand_pose} --flexdist 3.5 "

          if scoring_function == "vinardo":
            gnina_cmd += "--scoring vinardo --cnn_scoring none"

          gnina_cmd += f" -o {out_sdf} --seed 0 --exhaustiveness {ex}"
          obrms_cmd = f"obrms {out_sdf} {ligand_pose} > {rmsd_directory}/{rmsd_txt}"

          bash_lines.append(f"# ---------- Exhaustiveness {ex} ({mode}) {scoring_function} ----------")
          bash_lines.append(f"{gnina_cmd}")
          bash_lines.append(f"{obrms_cmd}\n")

# Write to file
with open("run_gnina_and_rmsd.sh", "w") as f:
    f.write("\n".join(bash_lines))

print("Generated run_gnina_and_rmsd.sh")


Generated run_gnina_and_rmsd.sh


In [ ]:
# take a look at the created file
!cat run_gnina_and_rmsd.sh

#!/bin/bash


export BABEL_DATADIR=/usr/share/openbabel/3.1.1

# ---------- Exhaustiveness 12 (flex) cnn ----------
./gnina -r protein_structures/7LME_fixed.pdb -l ligand_structures/Y6J_ideal.sdf --autobox_ligand ligand_structures/Y6J_corrected_pose.sdf --flexdist_ligand ligand_structures/Y6J_corrected_pose.sdf --flexdist 3.5  -o docking_results_/Y6J_e12_flex_cnn.sdf --seed 0 --exhaustiveness 12
obrms docking_results_/Y6J_e12_flex_cnn.sdf ligand_structures/Y6J_corrected_pose.sdf > rmsd_outputs_/rmsd_e12_flex_cnn.txt

# ---------- Exhaustiveness 12 (rigid) cnn ----------
./gnina -r protein_structures/7LME_fixed.pdb -l ligand_structures/Y6J_ideal.sdf --autobox_ligand ligand_structures/Y6J_corrected_pose.sdf -o docking_results_/Y6J_e12_rigid_cnn.sdf --seed 0 --exhaustiveness 12
obrms docking_results_/Y6J_e12_rigid_cnn.sdf ligand_structures/Y6J_corrected_pose.sdf > rmsd_outputs_/rmsd_e12_rigid_cnn.txt


In [ ]:
# run gnina!
! chmod +x run_gnina_and_rmsd.sh
!./run_gnina_and_rmsd.sh

              _             
             (_)            
   __ _ _ __  _ _ __   __ _ 
  / _` | '_ \| | '_ \ / _` |
 | (_| | | | | | | | | (_| |
  \__, |_| |_|_|_| |_|\__,_|
   __/ |                    
  |___/                     

gnina v1.3 master:97fa6bc+   Built Oct  3 2024.
gnina is based on smina and AutoDock Vina.
Please cite appropriately.

Commandline: ./gnina -r protein_structures/7LME_fixed.pdb -l ligand_structures/Y6J_ideal.sdf --autobox_ligand ligand_structures/Y6J_corrected_pose.sdf --flexdist_ligand ligand_structures/Y6J_corrected_pose.sdf --flexdist 3.5 -o docking_results_/Y6J_e12_flex_cnn.sdf --seed 0 --exhaustiveness 12
Flexible residues: A:25 A:41 A:46 A:49 A:142 A:145 A:163 A:165
Using random seed: 0

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************
Y6J | pose 0 | initial pose not within box

mode |  affinity  |  intramol  |    CNN     |   CNN
     | (kcal/

In [ ]:
# check the rmsd of flexible
!cat rmsd_outputs_/rmsd_e12_flex_cnn.txt

RMSD Y6J:6 ENGINEERED: YES 2.4156
RMSD Y6J:6 ENGINEERED: YES 4.91407
RMSD Y6J:6 ENGINEERED: YES 4.89473
RMSD Y6J:6 ENGINEERED: YES 4.42267
RMSD Y6J:6 ENGINEERED: YES 4.33595
RMSD Y6J:6 ENGINEERED: YES 8.53352
RMSD Y6J:6 ENGINEERED: YES 2.56349
RMSD Y6J:6 ENGINEERED: YES 5.79024
RMSD Y6J:6 ENGINEERED: YES 3.04516


In [ ]:
# check the rms of rigid
!cat rmsd_outputs_/rmsd_e12_rigid_cnn.txt

RMSD Y6J:6 ENGINEERED: YES 2.4156
RMSD Y6J:6 ENGINEERED: YES 4.91407
RMSD Y6J:6 ENGINEERED: YES 4.89473
RMSD Y6J:6 ENGINEERED: YES 4.42267
RMSD Y6J:6 ENGINEERED: YES 4.33595
RMSD Y6J:6 ENGINEERED: YES 8.53352
RMSD Y6J:6 ENGINEERED: YES 2.56349
RMSD Y6J:6 ENGINEERED: YES 5.79024
RMSD Y6J:6 ENGINEERED: YES 3.04516


In [ ]:
with open(f"docking_results_{label}/{ligand_id}_e12_flex_cnn.sdf") as f1, open(f"docking_results_{label}/{ligand_id}_e12_flex_cnn.sdf") as f2:
  poses_flex = f1.read()
  poses_rigid = f2.read()
  assert poses_flex == poses_rigid # if we don't see an error, the files are the same.

In [ ]:
import py3Dmol
v = py3Dmol.view()
v.addModel(open(f"{protein_directory}/{pdb_id}_fixed.pdb").read())
v.setStyle({'cartoon':{},'stick':{'radius':.1}})
v.addModel(open(f"{ligand_directory}/{ligand_id}_corrected_pose.sdf").read())
v.setStyle({'model':1},{'stick':{'colorscheme':'blueCarbon','radius':.125}})
v.addModelsAsFrames(open(f'{docked_dir}/{ligand_id}_e12_flex_cnn.sdf').read())
v.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})
v.zoomTo({'model':1})
v.rotate(270)
#v.animate({'interval':1000})


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
import py3Dmol
v = py3Dmol.view()
v.addModel(open(f"{protein_directory}/{pdb_id}_fixed.pdb").read())
v.setStyle({'cartoon':{},'stick':{'radius':.1}})
v.addModel(open(f"{ligand_directory}/{ligand_id}_corrected_pose.sdf").read())
v.setStyle({'model':1},{'stick':{'colorscheme':'blueCarbon','radius':.125}})
v.addModelsAsFrames(open(f'{docked_dir}/{ligand_id}_e12_rigid_cnn.sdf').read())
v.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})
v.zoomTo({'model':1})
v.rotate(270)
#v.animate({'interval':1000})


3Dmol.js failed to load for some reason. Please check your browser console for error messages.